### 计算给定数据集的信息熵，基于$H=-\sum_{i=1}^n p(x_i)log_2 p(x_i)$
#### 程序清单3-1

In [9]:
import numpy as np


def calShannonEnt(dataSet):
    """dataSet:二维array类型或者二维列表类型,包含最后一列分类属性"""
    num=len(dataSet)    # 总样本个数
    labelCounts={}    # key:属性名称，value:拥有该属性的样本所占的数量,len(labelCounts)即为分类数|y|
    for row in dataSet:    # 遍历每一行
        currentLabel=row[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel]=0    # 初始化键值对
        labelCounts[currentLabel]+=1
    
    # 计算香农熵
    shannonEnt=0.0    # 初始化
    for key in labelCounts.keys():
        prob=labelCounts[key]/num    # pk
        shannonEnt+=prob*np.log2(prob)
        
    return -shannonEnt

In [10]:
def createDataSet():
    dataSet = [
        [1,1,'yes'],
        [1,1,'yes'],
        [1,0,'no'],
        [0,1,'no'],
        [0,1,'no'],
    ]
    labels=['no surfacing','flippers']
    return dataSet, labels


myData=createDataSet()[0]
calShannonEnt(myData)

0.9709505944546686

#### 程序清单3-2 按照给定特征划分数据集

In [11]:
def splitDataSet(dataSet,axis,value):
    """挑选出axis所对应的特征为value的数据项"""
    retDataSet=[]    # 防止更改原列表，新建一个list
    for row in dataSet:
        if row[axis]==value:
            reducedFeatVec=row[:axis]
            reducedFeatVec.extend(row[axis+1:])    # 丢弃row[axis]
            retDataSet.append(reducedFeatVec)
            
    return retDataSet


myData

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [12]:
splitDataSet(myData,0,1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

#### 根据信息增益，选择最好的划分特征，基于：
#### $Gain(D,a)=Ent(D)-\sum_{v=1}^{V} \frac{|D^v|}{|D|}Ent(D^v)$

In [13]:
def chooseBestFeatureToSplit(dataSet):
    """分别计算每个特征划分后的Gain,以此选取最优特征作为划份特征
    返回最优特征对应的列索引"""
    numFeatures = len(dataSet[0])-1    # 注意，最后一列不是特征！！！（'yes','no'这一列！）
    baseEntropy=calShannonEnt(dataSet)    # Ent(D)
    
    bestInfoGain=0
    bestFeature=-1    # 初始化
    
    for i in range(numFeatures):    # 遍历每个特征，使得每个特征作为一个划分特征
        featureList=[row[i] for row in dataSet]
        uniqueVals=list(set(featureList))    # unique之,得到v
        newEntropy=0    # 记录公式中\sum项
        for value in uniqueVals:
            subDataSet=splitDataSet(dataSet,i,value)
            # 被除数需为浮点数
            newEntropy += calShannonEnt(subDataSet) * \
                (len(subDataSet)/float(len(dataSet)))

        infoGain=baseEntropy-newEntropy    # Gain
        
        if infoGain>bestInfoGain:
            bestInfoGain=infoGain    # 将最大的infoGain作为bestInfoGain
            bestFeature=i    # 且标记该infoGain对应的特征
    return bestFeature


chooseBestFeatureToSplit(myData)
# 结果表明，应首先选择第0个特征作为 划分特征

0

In [15]:
# 定义一个函数，类似于np.unique(arr,return_counts=True)
import operator    # operator.func返回一个可调用对象，如：s=operator.add,s(1,1)->2


def majorCnt(classList):
    """返回classList中种类对应数量最多的种类"""
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    
    sortedClassList=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassList[0][0]


majorCnt([0,0,1,1,1])

1

### 3.1.3 递归构建决策树
#### 程序清单3-4 创建数的函数代码

In [16]:
def creatTree(dataSet,labels):
    """
    dataSet: dataSet
    labels: dataSet中每个样本对应的标签,算法本身不需要该变量，
    将其也作为输入的原因是为了给出数据明确的含义
    
    return: 以字典数据结构保存的决策树
    """
    classList=[row[-1] for row in dataSet]    # 所有类的标签
    
    # 递归结束的条件1：classList中的类别完全相同，则停止划分
    if classList.count(classList[0])==len(classList):
        return classList[0]    # 叶子结点
    
    # 递归结束的基本条件2：遍历完所有的划分属性，但是仍无法将数据集划分成包含唯一一个类别的组
    # 此时返回classList中数量最多的类别作为最终类别
    if len(dataSet[0])==1:
        return majorCnt(classList)
    
    # 递归调用
    bestFeat=chooseBestFeatureToSplit(dataSet)    # 选择最好的特征用于划分子集
    bestFeatLabel=labels[bestFeat]    # 将索引值转化为有意义的类别（特征）名称
    
    myTree={bestFeatLabel:{}}    # 使用字典表示树
    del(labels[bestFeat])    # 用过的特征就不在用了
    
    featValues=[row[bestFeat] for row in dataSet]    # 最优特征对应的取值w,为一列
    uniqueVals=set(featValues)    # unique一下，为使用该特征进一步划分做准备
    
    for value in uniqueVals:
        sublabels=labels[:]    # 浅拷贝，此时的labels也是缩减过的,之前的del()函数
        myTree[bestFeatLabel][value]=creatTree(splitDataSet(dataSet,bestFeat,value),sublabels)    # splitDataSet
        # splitDataSet的过程中，减小了问题规模（dataSet的列数得到了缩减）
        # 只有在字典的键名为bestFeatLabel下面，才递归创建决策树
    return myTree


myDat,labels=createDataSet()
creatTree(myDat,labels)

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

### 3.4 示例：使用决策树预测隐形眼镜模型

In [17]:
fr = open('D:\\机器学习实战代码\\machinelearninginaction\\Ch03\\lenses.txt')
lenses=[row.strip().split('\t') for row in fr.readlines()]
lensesLabels=['age','prescript','astigmatic','tearRate']
lensesTree=creatTree(lenses,lensesLabels)

In [18]:
lensesTree

{'tearRate': {'reduced': 'no lenses',
  'normal': {'astigmatic': {'yes': {'prescript': {'myope': 'hard',
      'hyper': {'age': {'pre': 'no lenses',
        'young': 'hard',
        'presbyopic': 'no lenses'}}}},
    'no': {'age': {'pre': 'soft',
      'young': 'soft',
      'presbyopic': {'prescript': {'myope': 'no lenses',
        'hyper': 'soft'}}}}}}}}